# Chapter 6: Position Sizing


The objective of this Jupyter Notebook is to demonstrate and compare various position sizing algorithms for a quantitative trading strategy involving multiple autos stocks across multiple markets benchmarked to the S&P 500 (tickers: 7201.T, VOW3.DE, 005380.KS, RNO.PA, 1211.HK, F, GM, 7203.T, TSLA). 
It focuses on optimizing capital allocation based on trading signals, risk metrics, and volatility measures to maximize portfolio performance while managing drawdowns and exposures.

What we accomplish:

1. Retrieve and process historical price data, signals, and volatility metrics from ArcticDB.
2. Implement and simulate six position sizing strategies: Fixed Percentage, Fixed Risk, Raw Volatility-based, Standard Deviation-based, ATR-based, and Kelly Criterion.
3. Calculate key performance metrics (NAV, drawdowns, gross/net exposures) for each position sizing algorithm across different parameter levels.
4. Visualize and compare results to identify the most effective sizing method for the given signals and data, highlighting trade-offs between growth and risk.

In [ ]:
import math
import pandas as pd
import numpy as np
# import datetime as dt
# import time
import pathlib
import arcticdb as adb
from arcticdb import LibraryOptions
# from dateutil.relativedelta import relativedelta

import yfinance as yf
%matplotlib inline
import matplotlib.pyplot as plt
# import mplfinance as mpf

 ### **1. Import data from arcticDB and build DataFrames**

**1. Functions Definitions**

1. `initialise_adb_library_local(uri_path, library_name)`:
    - Initializes a local ArcticDB library using the LMDB (Lightning Memory-Mapped Database) storage engine.
    - Purpose: Sets up a local ArcticDB library for storing and retrieving time-series data.
    - Steps:
      - Constructs a URI for the LMDB storage using the provided `uri_path`.
      - Creates an ArcticDB instance using the URI.
      - Retrieves or creates a library with the specified `library_name` (if it doesn't already exist).
    

2. `adb_concat_single_column(library, symbols, column_name)`:
    - Purpose: Aggregates data (e.g., closing prices) from multiple symbols into a single DataFrame for analysis.
    - Concatenates a specific column from multiple symbols stored in an ArcticDB library into a single DataFrame.
    - Steps:
      - Filters the `symbols` list to include only those symbols that exist in the library.
      - Reads the specified `column_name` for each symbol and renames the column to the symbol's name.
      - Concatenates all the extracted columns into a single DataFrame.
    

3. `rohlc(df, relative=False)`:
    - Determines the column names for Open, High, Low, and Close (OHLC) prices in a DataFrame, optionally for relative prices.
    - Purpose: Dynamically identifies OHLC column names for both absolute and relative price data.
    - Steps:
      - Adds a prefix (`r`) to the column names if `relative=True`.
      - Checks if the DataFrame contains columns for OHLC prices (case-sensitive).
      - Returns the column names for OHLC prices or `np.nan` if not found.
    

In [ ]:
def initialise_adb_library_local(uri_path, library_name):
    uri = f"lmdb://{uri_path}" # this will set up the storage using the local file system
    ac = adb.Arctic(uri)
    library = ac.get_library(library_name, create_if_missing=True,)# library_options=LibraryOptions(dynamic_schema=True))
    return library

def adb_concat_single_column(library, symbols, column_name):
    symbols_list = [symbol for symbol in symbols if symbol in library.list_symbols()]
    comp_list_from_adb = [library.read(symbol, columns=[column_name]).data.rename(columns={column_name: symbol}) for symbol in symbols_list]
    return pd.concat(comp_list_from_adb, axis=1)

def rohlc(df,relative = False):
    if relative==True:
        rel = 'r'
    else:
        rel= ''      
    if 'Open' in df.columns:
       _o,_h,_l,_c = f'{rel}Open',f'{rel}High',f'{rel}Low',f'{rel}Close'      
    elif 'open' in df.columns:
        _o,_h,_l,_c = f'{rel}open',f'{rel}high',f'{rel}low',f'{rel}close'
    else:
        _o=_h=_l=_c= np.nan
    return _o,_h,_l,_c

def remove_duplicates(df, keeping='last'):
    return df.loc[:,~df.columns.duplicated(keep=keeping)]

**2. Retrieve data from arcticDB and generate dataframes**

1. Initialize ArcticDB Library:
    - `library_autos = initialise_adb_library_local('data', 'autos')`: Initializes a local ArcticDB library named `'autos'` stored in the `'data'` directory.
2. Retrieve Tickers List:
    - `tickers_list = list(library_autos.list_symbols())`: Retrieves a list of all symbols (tickers) stored in the `'autos'` library.
3. Define Currency Mapping:
    - `tickers_fx_dict = {...}`: Creates a dictionary mapping each ticker to its corresponding currency or exchange rate.
4.  Retrieve and Concatenate Data:
    - `px_df`, `px_df_rel`, `score_rel`: Retrieve the `'Close'`, `'rClose'` (relative close), `'score_rel'` (relative score) columns for all tickers and forward-fills missing values
5. Combine Data into a Single DataFrame:
    - Combines the absolute prices (`px_df`), relative prices (`px_df_rel`), and signals (`score_rel`) into a single DataFrame named `data`.
    - Set the boolean `look_ahead` to `True` 
    - if `look_ahead` is true, shift the `score_rel` df by +1 day to avoid lookahead bias
6. Plot Relative Scores:
    - Plots the last 252 rows of the `score_rel` DataFrame to visualize the relative scores over time.
7. Print `data` Shape

In [ ]:
library_autos = initialise_adb_library_local('data', 'autos')
tickers_list = list(library_autos.list_symbols())

tickers_fx_dict = {'7203.T': 'USDJPY', '7201.T': 'USDJPY', '1211.HK':'USDHKD', '005380.KS':'USDKRW',
                   'VOW3.DE' :'EURUSD', 'RNO.PA':'EURUSD', 'F':'local' ,'TSLA':'local' ,'GM':'local'  }

print('autos: ',len(library_autos.list_symbols()[:]), 'tickers_list:', tickers_list, end =', ')

px_df = adb_concat_single_column(library_autos, library_autos.list_symbols(), 'Close').ffill()
px_df_rel =  adb_concat_single_column(library_autos, library_autos.list_symbols(), 'rClose').ffill()
score_rel = adb_concat_single_column(library_autos, library_autos.list_symbols(), 'score_rel').ffill()
look_ahead = True
if look_ahead == False:
    score_rel = score_rel.shift(+1) # shift to avoid lookahead bias

data = pd.concat([px_df.add_suffix('_abs'), px_df_rel.add_suffix('_rel'), score_rel.add_suffix('_signal')], axis=1)
score_rel[-252:].plot(figsize=(15,3), grid = True)
print("data shape:",data.shape, 'look_ahead:', look_ahead)

**3. Convert Local price to USD**

1. Iterates through `tickers_list`: For each ticker, it retrieves the corresponding data from the `library_autos` ArcticDB library.
2. Currency Conversion: Divides the `Close` price of the ticker by its associated currency (`ccy`) from `tickers_fx_dict` to calculate the price in a common currency, rounding to 4 decimal places.
3. Updates `data` DataFrame: Adds the converted prices as new columns in the `data` DataFrame with the suffix `_fx`.
4. Removes Duplicate Columns: Ensures no duplicate columns exist in `data`.
5. Handles Missing Data: Forward-fills missing values and drops rows with all `NaN` values.
6. Displays Last Rows: Prints the last 3 rows and 10 columns of the updated `data` DataFrame.

In [ ]:
for ticker in tickers_list:
    df = library_autos.read(ticker).data
    ccy = tickers_fx_dict[ticker]
    px_ccy = round(df['Close'].div(df[ccy]),4)
    data[f'{ticker}_fx'] = px_ccy
data = remove_duplicates(data, keeping='last')
data = data.ffill().dropna(how='all', axis=0)

data.iloc[-3:,-10:]

In [ ]:
data_copy = data.copy()

**4. Scores and relative series for each stock in the investment universe**

This code iterates through each ticker in the `tickers_list` and performs the following steps:

1. Retrieve Data: For each `ticker`, it reads the associated data from the `library_autos` ArcticDB library and stores it in the `df` variable.
2. Plot Data: It plots the last 1500 rows of the columns `rClose`, `score`, `score_rel`, and `score_abs` from the `df` DataFrame:
    - `rClose` is plotted on the primary y-axis.
    - `score`, `score_rel`, and `score_abs` are plotted on the secondary y-axis.
3. Set Titles and Labels:
    - The plot is titled with the format `'{ticker}: rClose and scores'`.
    - The primary y-axis is labeled as `'rClose'`.
    - The secondary y-axis is labeled as `'Scores'`.
4. Access Secondary Y-Axis: The secondary y-axis (`right_ax`) is accessed and stored for further customization if needed.

In [ ]:
for ticker in tickers_list:
    df = library_autos.read(ticker).data    
    ax = df[-1500:][['rClose', 'score', 'score_rel','score_abs']].plot(figsize=(15, 3), secondary_y = ['score', 'score_rel','score_abs'], 
                            grid = True, style = ['k', 'm', 'c','y'], title=f'{ticker}: rClose and scores')
    ax.set_ylabel('rClose')    
    ax.right_ax = ax.get_figure().axes[1]
    ax.right_ax.set_ylabel('Scores')

### **2 Position Sizing Algorithms**

**2.1 Position Sizing Functions Explanations**

1. `size_limit(nav_pct, limit_pct)`:  Clips the position size percentage (`nav_pct`) to a specified limit (`limit_pct`), ensuring it stays within the range [-limit_pct, limit_pct]. This prevents over-allocation beyond the allowed bounds.

2. `nav_allocation(nav, nav_pct)`:  Calculates the dollar amount allocated to a position by multiplying the net asset value (`nav`) by the position size percentage (`nav_pct`).

3. `nav_signal(nav_pct, nav, signal)`:  Computes the signal-adjusted NAV allocation by multiplying `nav_pct`, `nav`, and the trading signal (`signal`). Replaces any NaN values with 0 to handle missing data.

4. `shares_target(target_mv, fraction)`:  Determines the target number of shares based on the target market value (`target_mv`) divided by the price or fraction (`fraction`). Returns 0 if `fraction` is NaN, zero, or if the result is infinite or NaN.

5. `round_lot(raw_shares, lot_size)`:  Rounds the raw number of shares (`raw_shares`) down to the nearest multiple of the lot size (`lot_size`), preserving the original sign (positive or negative).

6. `trading_value(round_lots, price)`:  Calculates the total trading value by multiplying the rounded lots (`round_lots`) by the price (`price`).

7. `trading_value_local(round_lots, price, fx)`:  Calculates the trading value in local currency by multiplying the rounded lots (`round_lots`) by the price (`price`) and the exchange rate (`fx`).

8. `calculate_drawdown(nav_series)`:  Computes the drawdown series for a NAV series by finding the cumulative maximum (peak) and calculating the percentage decline from that peak. Returns the drawdown as a percentage.

In [ ]:
def size_limit(nav_pct, limit_pct):
    return nav_pct.clip(-limit_pct, limit_pct)

def nav_allocation(nav, nav_pct): 
    return nav * nav_pct

def nav_signal(nav_pct, nav, signal):
    signal_nav = np.multiply(nav_pct, np.multiply(nav, signal))
    return np.nan_to_num(signal_nav) 

def shares_target(target_mv, fraction):
    if pd.isna(fraction) or fraction == 0:
        return 0  
    shares = target_mv / fraction
    if math.isinf(shares) or pd.isna(shares):
        return 0 
    return shares

def round_lot(raw_shares, lot_size):
    return math.floor(abs(raw_shares) / lot_size) * lot_size * np.sign(raw_shares)

def trading_value(round_lots, price):
    return round_lots * price

def trading_value_local(round_lots, price, fx):
    return round_lots * price * fx

def calculate_drawdown(nav_series):
    peak = nav_series.cummax()
    drawdown = (nav_series - peak) / peak  
    return drawdown


**2.2 Simulate Position Sizing**

The `simulate_position_sizing` function simulates a trading strategy's performance over time for multiple tickers, incorporating position sizing, commissions, and P&L calculations. It iterates through each date in `data.index[2:]` and each ticker in `tickers_list`, adjusting positions based on signals and fraction types, then updates NAV, long/short exposures, and returns lists of these values.

Key steps per date and ticker:
- Retrieve current (`price_fx`) and previous (`price_fx_prev`) prices, and the trading signal.
- Compute `fraction` based on `fraction_type`:
    - 1: `fraction = price_fx`
    - 2: `fraction = price_fx * constant`
    - 3: `fraction = price_fx * data.at[t, f'{ticker}{ratio}']`
    - 4: `fraction = data.at[t, f'{ticker}{ratio}']`
- Calculate target market value (`target_mv`) using `nav_signal(pct, nav, signal)`.
- Determine shares to trade (`shares`) via `shares_target(target_mv, fraction)`.
- Round shares to `lot_size` using `round_lot`, then compute `round_lots` as the difference from current position.
- Calculate trade value and commission, update position and cash balance.
- Compute daily P&L for the ticker and accumulate total daily P&L.
- Track long/short market values based on position sign.
- After all tickers, update NAV with total daily P&L and append to lists.

Note: The function assumes `nav_signal`, `shares_target`, `round_lot`, and `trading_value` are defined elsewhere (e.g., in earlier cells). It does not handle look-ahead bias in signals, as noted in the markdown. The commented-out P&L calculation before position update is inactive; the active code calculates it after.

In [ ]:
def simulate_position_sizing(start_K, lot_size, com_rate, pct, look_ahead,fraction_type, constant= None, ratio='_2std'):
    # for pct in pct_list:
    nav = start_K
    cash_balance = start_K
    position_dict = {ticker: 0 for ticker in tickers_list}  # Tracks positions for each ticker
    pl_dict = {ticker: 0 for ticker in tickers_list}  # Tracks P&L for each ticker
    # daily_pl_sum = 0  # Tracks total daily P&L across all tickers
    nav_list = []  ;    long_mv_list = []   ;   short_mv_list = []   # Tracks NAV over time

    for t in data.index[2:]:  # Loop through each bar
        daily_pl_sum = 0 ; long_mv = 0 ; short_mv = 0  # Reset at every bar
        for ticker in tickers_list[:]:  # Loop through each ticker
            price_fx = data.at[t,f'{ticker}_fx']
            price_fx_prev = data[f'{ticker}_fx'].shift(1).at[t]
            signal = data.at[t,f'{ticker}_signal']
            
            # Compute fraction based on type
            if fraction_type == 1:
                fraction = price_fx
            elif fraction_type == 2:
                fraction = price_fx * constant
            elif fraction_type == 3:
                fraction = price_fx *  data.at[t,f'{ticker}{ratio}']
            elif fraction_type == 4:
                fraction = data.at[t,f'{ticker}{ratio}']
            else:
                fraction = price_fx  # default to option 1

            # Calculate position size percentage,target market value and shares
            target_mv = nav_signal(pct, nav,  signal)
            shares = shares_target(target_mv, fraction)            
            
            # # Calculate daily P&L
            if not look_ahead:
                daily_pl = position_dict[ticker] * (price_fx - price_fx_prev)
                pl_dict[ticker] += daily_pl
                daily_pl_sum += daily_pl

                # Calculate theoretical lots and round lots:
            theoretical_lots = round_lot(shares, lot_size)
            round_lots = theoretical_lots - position_dict[ticker]

            # Calculate trade value and commission:
            trade_value = trading_value(round_lots, price_fx)
            commission = abs(trade_value) * com_rate

            # Update position and cash balance
            position_dict[ticker] += round_lots
            cash_balance -= (trade_value + commission)            
            
            # Calculate daily P&L
            if look_ahead:            
                daily_pl = position_dict[ticker] * (price_fx - price_fx_prev)
                pl_dict[ticker] += daily_pl
                daily_pl_sum += daily_pl
            
            # Calculate Long and Short exposure
            if position_dict[ticker] > 0:  # Long exposure
                long_mv += position_dict[ticker] * price_fx
            elif position_dict[ticker] < 0:  # Short exposure
                short_mv += position_dict[ticker] * price_fx

        # Update NAV
        nav += daily_pl_sum
        nav_list.append(nav)
        long_mv_list.append(long_mv)
        short_mv_list.append(short_mv)
    return nav_list, long_mv_list, short_mv_list

**2.3 Fixed Dollar/Percentage**

2.1 The section refers to a position-sizing technique where the size of a trade is determined based on a fixed dollar amount or a fixed percentage of the available capital.

#### Key Concepts:
1. **Fixed Dollar Position Sizing**:
    - A fixed dollar amount is allocated to each trade, regardless of the account size or market conditions.
    - For example, if the fixed dollar amount is $10,000, every trade will use exactly $10,000 worth of capital.

2. **Fixed Percentage Position Sizing**:
    - A fixed percentage of the account's net asset value (NAV) is allocated to each trade.
    - For example, if the fixed percentage is 2% and the NAV is $100,000, the trade size will be $2,000.

- The `pct_list` stores position sizes for different fixed percentages (e.g., 2%, 5%, 10%, 25%) and their adjusted values based on relative scores (`score_rel`).

This code block implements a **fixed percentage position sizing** strategy for a trading simulation across multiple tickers. It simulates portfolio performance for different fixed percentages of NAV allocated to each position, calculates key metrics like NAV, drawdowns, and exposures, and visualizes the results.

#### Key Components:
1. **Initialization**:
    - `start_K = 1000000`: Starting NAV (Net Asset Value) of $1,000,000.
    - `lot_size = 100`: Minimum trade size (e.g., shares must be multiples of 100).
    - `com_rate = 0.001`: Commission rate (0.1% per trade).
    - `pct_list = [0.02, 0.05, 0.1, 0.2]`: List of fixed percentages (2%, 5%, 10%, 20%) of NAV to allocate per position.
    - `nav_df`: A new pandas DataFrame with index matching `data.index[2:]` (skipping the first two rows, likely for data alignment).
    - `algo = 'fix_pct_'`: Prefix for column names to indicate the algorithm type.
    - Column name lists (`nav_cols`, `gross_exposure_cols`, `net_exposure_cols`) are defined but not directly used in this block (possibly for reference or later use).

2. **Simulation Loop**:
    - For each `pct` in `pct_list`:
      - Calls `simulate_position_sizing(start_K, lot_size, com_rate, pct, look_ahead, fraction_type=1, constant=None, ratio=None)`:
         - This function (defined earlier in the notebook) simulates trading over time for all tickers in `tickers_list`.
         - `fraction_type=1`: Uses the current price (`price_fx`) as the fraction for share calculation (i.e., fixed dollar allocation scaled by price).
         - Returns lists: `nav_list` (NAV over time), `long_mv_list` (long market value), `short_mv_list` (short market value).
      - Adds columns to `nav_df`:
         - `nav_{algo}_{pct}`: NAV series.
         - `longMV_{algo}_{pct}`: Long market value.
         - `shortMV_{algo}_{pct}`: Short market value.
         - `drawdown_{algo}_{pct}`: Drawdown (calculated using `calculate_drawdown`, which computes peak-to-trough declines).
         - `net_{algo}_{pct}`: Net exposure = (long MV + short MV) / NAV (measures directional bias).
         - `gross_{algo}_{pct}`: Gross exposure = (long MV - short MV) / NAV (measures total exposure).

3. **Visualization**:
    - Plots NAV series for all `pct` values (from row 500 onward to skip initial data):
      - Primary y-axis: NAV values.
      - Secondary y-axis: NAV for the last `pct` (0.2), which may be redundant but highlights the highest allocation.
    - Plots drawdowns, gross exposure, and net exposure for all `pct` values.
    - All plots use `figsize=(15, 5)`, grid, and titles for clarity.

4. **Output**:
    - `nav_df.columns`: Prints the list of all columns in `nav_df` (e.g., for verification).

### Notes:
- This strategy allocates a fixed percentage of NAV to each position based on signals, without considering volatility or stop-losses (unlike fixed risk sizing).
- The simulation accounts for commissions, lot sizes, and signals from `score_rel`.
- `look_ahead` (a boolean variable) affects whether signals are shifted to avoid lookahead bias.
- Results show how higher percentages (e.g., 20%) lead to higher NAV growth but also greater drawdowns and exposures.
- No new variables or imports are introduced; it builds on existing ones like `data`, `tickers_list`, and functions from prior cells.

In [ ]:
start_K = 1000000 ; lot_size = 100 ; com_rate = 0.001
pct_list = [0.02 ,0.05, 0.1, 0.2]
nav_df = pd.DataFrame(index=data.index[2:])

# Fixed Percentage
algo = 'fix_pct'
nav_cols = [f'nav_{algo}_{pct}' for pct in pct_list]
gross_exposure_cols = [f'gross_{algo}_{pct}' for pct in pct_list]
net_exposure_cols = [f'net_{algo}_{pct}' for pct in pct_list]

for pct in pct_list:
    nav_list, long_mv_list, short_mv_list = simulate_position_sizing(start_K, lot_size, com_rate, 
                                                    pct, look_ahead, fraction_type = 1,  constant= None, ratio= None)
    nav_df[f'nav_{algo}_{pct}'] = nav_list
    nav_df[f'longMV_{algo}_{pct}'] = long_mv_list
    nav_df[f'shortMV_{algo}_{pct}'] = short_mv_list

    nav_df[f'drawdown_{algo}_{pct}'] = calculate_drawdown(nav_df[f'nav_{algo}_{pct}'])
    nav_df[f'net_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] + nav_df[f'shortMV_{algo}_{pct}']) / (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}'])
    nav_df[f'gross_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) / nav_df[f'nav_{algo}_{pct}']


nav_df.dropna().filter(like=f'nav_{algo}').plot(figsize=(15, 5), grid=True, secondary_y=[f'nav_{algo}{pct}'], title=f'NAV - Fixed Percentage: {pct_list}')
nav_df.dropna().filter(like=f'drawdown_{algo}').plot(figsize=(15, 5), grid=True, title=f'Drawdowns - Fixed Percentage: {pct_list}')
nav_df.dropna().filter(like=f'gross_{algo}').plot(figsize=(15, 5), grid=True, title=f'Gross Exposure - Fixed Percentage: {pct_list}')
nav_df.dropna().filter(like=f'net_{algo}').plot(figsize=(15, 5), grid=True, title=f'Net Exposure - Fixed Percentage: {pct_list}')
nav_df.columns

**2.4 Fixed Risk Position Sizing**

This block of code implements a **fixed risk position sizing** strategy. It calculates the position size for each ticker based on a fixed percentage of the portfolio's NAV (Net Asset Value) and the stop-loss distance (`distance_stop_loss`). The key difference from **fixed percentage position sizing** is that the position size is determined by the risk per trade, which depends on the stop-loss distance (`dsl`) rather than just the NAV percentage.

##### Key Steps:
1. Initialization:
    - `distance_stop_loss` defines the stop-loss as 10% of the current price.
    - `risk_list` contains different risk percentages (e.g., 0.25%, 0.5%, etc.).
    - Tracks NAV, cash balance, positions, and P&L for each ticker.
2. Position Sizing:
    - For each ticker, the position size is calculated based on the risk percentage (`risk`) and the stop-loss distance (`dsl`).
    - `target_mv` is the capital allocated to the trade, and `shares` is the number of shares determined by dividing `target_mv` by `dsl`.
3. Trade Execution:
    - Calculates the number of shares to trade (`round_lots`), trade value, and commission.
    - Updates positions, cash balance, and daily P&L.
4. NAV Update:
    - Updates NAV by adding daily P&L and tracks long/short exposures.
5. Visualization:
    - Plots NAV for different risk percentages.

***Difference from Fixed Percentage:***
- *Fixed Percentage*: Allocates a fixed percentage of NAV to each trade, regardless of stop-loss or volatility.
- *Fixed Risk*: Allocates capital based on the risk per trade, considering the distance to stop-loss. This ensures that the maximum loss per trade is capped at the specified risk percentage. This uses a lot more capital. Conventional wisdom caps Fixed fraction at 2% of capital.

In [ ]:
pct_list = [0.005 ,0.01, 0.02, 0.05]

algo = 'fix_risk'
nav_cols = [f'nav_{algo}_{pct}' for pct in pct_list]
gross_exposure_cols = [f'gross_{algo}_{pct}' for pct in pct_list]
net_exposure_cols = [f'net_{algo}_{pct}' for pct in pct_list]
dsl = 0.1

for pct in pct_list:
    nav_list, long_mv_list, short_mv_list = simulate_position_sizing(start_K, lot_size, com_rate, pct, look_ahead,
                                                                     fraction_type= 2, constant= dsl, ratio=None)

    nav_df[f'nav_{algo}_{pct}'] = nav_list
    nav_df[f'longMV_{algo}_{pct}'] = long_mv_list
    nav_df[f'shortMV_{algo}_{pct}'] = short_mv_list

    nav_df[f'drawdown_{algo}_{pct}'] = calculate_drawdown(nav_df[f'nav_{algo}_{pct}'])
    nav_df[f'net_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] + nav_df[f'shortMV_{algo}_{pct}']) / (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}'])
    nav_df[f'gross_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) / nav_df[f'nav_{algo}_{pct}']

nav_df.dropna().filter(like=f'nav_{algo}').plot(figsize=(15, 5), grid=True, secondary_y = [f'nav_{algo}_{pct}'], title=f'NAV - Fixed Risk dsl= {dsl}: {pct_list}')
nav_df.dropna().filter(like=f'drawdown_{algo}').plot(figsize=(15, 5), grid=True, title = f'Drawdowns - Fixed Risk: {pct_list}')
nav_df.dropna().filter(like=f'gross_{algo}').plot(figsize=(15, 5), grid=True, title = f'Gross Exposure - Fixed Risk dsl= {dsl}: {pct_list}')
nav_df.dropna().filter(like=f'net_{algo}').plot(figsize=(15, 5), grid=True, title = f'Net Exposure - Fixed Risk dsl= {dsl}: {pct_list}')

##### **2.4 Volatility Position Sizing**

These functions calculate volatility metrics for quantitative finance, using pandas and numpy.

1. **`target_vol_daily(annual_target, len_tickers)`**  
    - Computes the daily target volatility per ticker, given a portfolio-level target volatility (`target`) and the number of tickers (`len_tickers`). It adjusts for annualization and diversification.
    - Formula: `annual_target / sqrt(252) / sqrt(len_tickers)`.  
    - Output: Float for daily volatility scaling.

2. **`average_true_range(df, _h, _l, _c, n)`**  
    - Calculates the Average True Range (ATR) over a rolling window of `n` periods. ATR measures market volatility by considering the range between the high, low, and previous close prices.
    - Inputs: DataFrame `df`, column names `_h`, `_l`, `_c`, window `n`.  
    - Output: Series of ATR values.

3. **`raw_log_volatility(df, _c, n)`**  
    - Calculates the rolling standard deviation of daily logarithmic returns over `n` periods, representing realized volatility.
    - Inputs: DataFrame `df`, column name `_c`, window `n`.  
    - Output: Series of volatility values.  

Used for risk management in position sizing.

In [ ]:
def target_vol_daily(annual_target, len_tickers):
    daily_target = annual_target / np.sqrt(252)
    target_vol_daily = daily_target / np.sqrt(len_tickers)
    return target_vol_daily

def average_true_range(df, _h, _l, _c, n):
    atr =  (df[_h].combine(df[_c].shift(), max) - df[_l].combine(df[_c].shift(), min)).rolling(window=n).mean()
    return atr

def raw_log_volatility(df, n):
    daily_log_returns = np.log(df / df.shift(1))
    log_realised = daily_log_returns.rolling(n).std(ddof=0) 
    return log_realised

This code calculates and visualizes realized volatility for relative prices:

- `n = 21`: Sets the rolling window to 21 days.
- `raw_vol = raw_log_volatility(px_df_rel, n)`: Computes the rolling standard deviation of daily log returns for each ticker in `px_df_rel` over 21 days, representing realized volatility.
- `ax = raw_vol[-500:].plot(...)`: Plots the last 500 data points of `raw_vol` with a 16x3 figure size, grid, and title "Realised Volatility over 21 days".
- `ax.legend(loc='lower left')`: Positions the legend in the lower left.
- `plt.show()`: Displays the plot.

In [ ]:
n =21
raw_vol = raw_log_volatility(px_df_rel, n)
raw_vol = raw_vol.ffill().dropna(how='all', axis=0).round(4)
ax =raw_vol[-500:].plot(figsize=(16,3), grid=True, title=f'Realised Volatility over {n} days')
ax.legend(loc='lower left')
plt.show()

This code calculates the rolling standard deviation of relative prices (`px_df_rel`) over a 21-day window (`n`), then divides it by the relative prices to get a normalized volatility measure (coefficient of variation-like). It plots the last 500 data points of this `stdev` DataFrame with a title, grid, and legend.

In [ ]:
stdev_df = px_df_rel.rolling(n).std(ddof=0).div(px_df_rel)
stdev_df = stdev_df.ffill().dropna(how='all', axis=0).round(4)
ax =stdev_df[-500:].plot(figsize=(16,3), grid=True, title=f'Standard Deviation over {n} days')
ax.legend(loc='upper left')
plt.show()


This code calculates and visualizes the Average True Range (ATR) normalized by the closing price for each ticker over a 21-day rolling window, representing relative volatility risk.

- Initializes an empty DataFrame `atr_df` with the same index as `data`.
- Retrieves relative OHLC column names (`_o`, `_h`, `_l`, `_c`) using `rohlc(df, relative=True)`.
- For each ticker in `tickers_list`:
    - Reads the ticker's data from `library_autos`.
    - Computes ATR using `average_true_range(df, _h, _l, _c, n)` and normalizes it by dividing by the close price (`df[_c]`), storing as `atr_risk`.
    - Adds `atr_risk` as a new column in `atr_df` (e.g., `'005380.KS_atr'`).
- Forward-fills missing values in `atr_df` and drops rows where all values are NaN.
- Plots the last 500 rows of `atr_df` with a 16x3 figure, grid, title "ATR over 21 days", and legend in the upper left corner, then displays the plot.

In [ ]:
atr_df = pd.DataFrame(index=data.index)  
_o,_h,_l,_c = rohlc(df, relative=True)

for ticker in tickers_list[:]:
    df = library_autos.read(ticker).data
    atr_risk = average_true_range(df, _h, _l, _c, n ).div(df[_c])
    atr_df[f'{ticker}_atr'] = atr_risk
atr_df = atr_df.ffill().dropna(how='all', axis=0).round(4)

ax = atr_df[-500:].plot(figsize=(16,3), grid=True, title=f'ATR over {n} days')
ax.legend(loc='upper left')
plt.show()

atr_df.tail()

This code concatenates additional DataFrames to the existing `data` DataFrame along the columns (axis=1), adding suffixes to avoid naming conflicts: `raw_vol` with '_raw_vol', `stdev_df` with '_std', and `atr_df` (which already has suffixes like '_atr'). It then removes duplicate columns, keeping the last occurrence, and displays the last 5 rows of the updated `data` DataFrame. This integrates volatility metrics into the main dataset for further analysis.

In [ ]:
data =pd.concat([data, raw_vol.add_suffix('_raw_vol'), stdev_df.add_suffix('_std'), atr_df], axis=1)
data = remove_duplicates(data, keeping='last')
data.tail()

This code implements a **Raw Volatility-based position sizing** strategy for the trading simulation. It calculates daily target volatilities for each ticker based on annual targets (5%, 10%, 15%, 20%), adjusted for the number of tickers (9) and annualized to daily using `target_vol_daily`. For each target volatility level, it simulates trading using `fraction_type=3` and `ratio='_raw_vol'`, where position sizes are scaled by the raw log volatility of relative prices. It stores NAV, long/short market values, drawdowns, net exposure, and gross exposure in `nav_df`, then plots these metrics over time for comparison across volatility levels. Finally, it prints the column names of `nav_df`.

1. **Calculate Daily Target Volatilities**:  
    Compute daily target volatilities for each ticker using annual targets (5%, 10%, 15%, 20%). Adjust for the number of tickers (9) and annualize to daily values with the `target_vol_daily` function.

2. **Prepare Volatility Ratios**:  
    Store the calculated daily volatilities in `pct_list` for use as position sizing parameters.

3. **Simulate Trading for Each Volatility Level**:  
    For each volatility level in `pct_list`, run the `simulate_position_sizing` function with:  
    - `fraction_type=3` (scales position size by price multiplied by volatility).  
    - `ratio='_raw_vol'` (uses raw log volatility of relative prices for scaling).  
    - Other parameters: `start_K`, `lot_size`, `com_rate`, `look_ahead`, etc.

4. **Store Simulation Results**:  
    Append the following to `nav_df` for each volatility level:  
    - NAV series (`nav_{algo}_{pct}`).  
    - Long market value (`longMV_{algo}_{pct}`).  
    - Short market value (`shortMV_{algo}_{pct}`).  
    - Drawdown (`drawdown_{algo}_{pct}`, calculated via `calculate_drawdown`).  
    - Net exposure (`net_{algo}_{pct}` = (long MV + short MV) / (long MV - short MV)).  
    - Gross exposure (`gross_{algo}_{pct}` = (long MV - short MV) / NAV).

5. **Visualize Results**:  
    Plot NAV, drawdowns, gross exposure, and net exposure for all volatility levels using `figsize=(15, 5)`, grid, and titles. Include secondary y-axis for the last volatility level (0.0042).

6. **Output Column Names**:  
    Print the list of all columns in `nav_df` for verification.

In [ ]:
annual_target_list = [0.05, 0.1, 0.15, 0.2]
len_tickers = len(tickers_list)
pct_list = [round(target_vol_daily(annual_target, len_tickers),4) for annual_target in annual_target_list]

# Raw Volatility
algo = 'raw_vol'
nav_cols = [f'nav_fix_pct{pct}' for pct in pct_list]
gross_exposure_cols = [f'gross_{algo}_{pct}' for pct in pct_list]
net_exposure_cols = [f'net_{algo}_{pct}' for pct in pct_list]

for pct in pct_list:
    nav_list, long_mv_list, short_mv_list = simulate_position_sizing(start_K, lot_size, com_rate, pct, look_ahead,
                                                                     fraction_type= 3, constant= None, ratio='_raw_vol')
    nav_df[f'nav_{algo}_{pct}'] = nav_list
    nav_df[f'longMV_{algo}_{pct}'] = long_mv_list
    nav_df[f'shortMV_{algo}_{pct}'] = short_mv_list

    nav_df[f'drawdown_{algo}_{pct}'] = calculate_drawdown(nav_df[f'nav_{algo}_{pct}'])
    nav_df[f'net_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] + nav_df[f'shortMV_{algo}_{pct}']) / (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) 
    nav_df[f'gross_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) / nav_df[f'nav_{algo}_{pct}']

nav_df.dropna().filter(like=f'nav_{algo}').plot(figsize=(15, 5), grid=True, secondary_y = [f'nav_{algo}_{pct}'], title=f'NAV - Raw Volatility: {annual_target_list}')
nav_df.dropna().filter(like=f'drawdown_{algo}').plot(figsize=(15, 5), grid=True, title = f'Drawdowns - Raw Volatility: {annual_target_list}')
nav_df.dropna().filter(like=f'gross_{algo}').plot(figsize=(15, 5), grid=True, title = f'Gross Exposure - Raw Volatility: {annual_target_list}')
nav_df.dropna().filter(like=f'net_{algo}').plot(figsize=(15, 5), grid=True, title = f'Net Exposure - Raw Volatility: {annual_target_list}')
nav_df.columns

This code implements a standard deviation-based position sizing strategy for the trading simulation, where position sizes are scaled by the rolling standard deviation of relative prices (using `ratio='_std'` and `fraction_type=3`) to adjust for volatility across different target volatility levels (from `pct_list`, corresponding to annual targets like 5%, 10%, 15%, 20%). For each level, it runs the simulation via `simulate_position_sizing`, stores NAV, long/short market values, and computes drawdowns, net exposure (long MV + short MV) / (long MV - short MV), and gross exposure (long MV - short MV) / NAV in `nav_df`, then visualizes these metrics over time with plots for NAV (with secondary y-axis), drawdowns, gross exposure, and net exposure, and finally lists all columns in `nav_df` for verification.

In [ ]:
# Standard Deviation
algo = 'std'
nav_cols = [f'nav_fix_pct{pct}' for pct in pct_list]
gross_exposure_cols = [f'gross_{algo}_{pct}' for pct in pct_list]
net_exposure_cols = [f'net_{algo}_{pct}' for pct in pct_list]

for pct in pct_list:
    nav_list, long_mv_list, short_mv_list = simulate_position_sizing(start_K, lot_size, com_rate, pct, look_ahead,
                                                                     fraction_type= 3, constant= None, ratio='_std')
    nav_df[f'nav_{algo}_{pct}'] = nav_list
    nav_df[f'longMV_{algo}_{pct}'] = long_mv_list
    nav_df[f'shortMV_{algo}_{pct}'] = short_mv_list

    nav_df[f'drawdown_{algo}_{pct}'] = calculate_drawdown(nav_df[f'nav_{algo}_{pct}'])
    nav_df[f'net_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] + nav_df[f'shortMV_{algo}_{pct}']) / (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) 
    nav_df[f'gross_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) / nav_df[f'nav_{algo}_{pct}']

nav_df.dropna().filter(like=f'nav_{algo}').plot(figsize=(15, 5), 
                                                grid=True, secondary_y = [f'nav_{algo}_{pct}'], title=f'NAV - Standard Deviation: {annual_target_list}')
nav_df.dropna().filter(like=f'drawdown_{algo}').plot(figsize=(15, 5), grid=True, title = f'Drawdowns - Standard Deviation: {annual_target_list}')
nav_df.dropna().filter(like=f'gross_{algo}').plot(figsize=(15, 5), grid=True, title = f'Gross Exposure - Standard Deviation: {annual_target_list}')
nav_df.dropna().filter(like=f'net_{algo}').plot(figsize=(15, 5), grid=True, title = f'Net Exposure - Standard Deviation: {annual_target_list}')
nav_df.columns

This code implements an Average True Range (ATR)-based position sizing strategy for the trading simulation. It calculates daily target volatilities from annual targets (5%, 10%, 15%, 20%), adjusted for the number of tickers (9). For each volatility level, it runs the simulation using `fraction_type=3` (scaling position size by price multiplied by ATR) and `ratio='_atr'` (using ATR as the volatility metric). Results (NAV, long/short market values, drawdowns, net exposure, and gross exposure) are stored in `nav_df`, then visualized with plots for NAV, drawdowns, gross exposure, and net exposure across the levels. Finally, it lists the `nav_df` columns. This approach adjusts allocations based on ATR to manage volatility, similar to raw volatility sizing but using a different measure.

In [ ]:
annual_target_list = [0.05, 0.1, 0.15, 0.2]
len_tickers = len(tickers_list)
pct_list = [round(target_vol_daily(annual_target, len_tickers),4) for annual_target in annual_target_list]

# Average True Range
algo = 'atr'
nav_cols = [f'nav_fix_pct{pct}' for pct in pct_list]
gross_exposure_cols = [f'gross_{algo}_{pct}' for pct in pct_list]
net_exposure_cols = [f'net_{algo}_{pct}' for pct in pct_list]

for pct in pct_list:
    nav_list, long_mv_list, short_mv_list = simulate_position_sizing(start_K, lot_size, com_rate, pct, look_ahead,
                                                                     fraction_type= 3, constant= None, ratio='_atr')

    nav_df[f'nav_{algo}_{pct}'] = nav_list
    nav_df[f'longMV_{algo}_{pct}'] = long_mv_list
    nav_df[f'shortMV_{algo}_{pct}'] = short_mv_list

    nav_df[f'drawdown_{algo}_{pct}'] = calculate_drawdown(nav_df[f'nav_{algo}_{pct}'])
    nav_df[f'net_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] + nav_df[f'shortMV_{algo}_{pct}']) / (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) 
    nav_df[f'gross_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) / nav_df[f'nav_{algo}_{pct}']

nav_df.dropna().filter(like=f'nav_{algo}').plot(figsize=(15, 5), 
                                                grid=True, secondary_y = [f'nav_{algo}_{pct}'], title=f'NAV - ATR: {annual_target_list}')
nav_df.dropna().filter(like=f'drawdown_{algo}').plot(figsize=(15, 5), grid=True, title = f'Drawdowns - ATR: {annual_target_list}')
nav_df.dropna().filter(like=f'gross_{algo}').plot(figsize=(15, 5), grid=True, title = f'Gross Exposure - ATR: {annual_target_list}')
nav_df.dropna().filter(like=f'net_{algo}').plot(figsize=(15, 5), grid=True, title = f'Net Exposure - ATR: {annual_target_list}')
nav_df.columns

**2.5 Kelly Criterion**

- **`kelly_fraction(returns)`**: Calculates the Kelly fraction for a series of returns, which is the optimal fraction of capital to invest to maximize long-term growth. It computes win probability (W), average win, average loss, win/loss ratio (R), and Kelly fraction as max(W - (1 - W) / R, -1) if R > 0, else 0. This caps the fraction at -1 to avoid extreme shorting.

- **`rolling_function(returns, function, f_duration)`**: Applies a given function (e.g., `kelly_fraction`) over a rolling window of `f_duration` periods on the returns series, returning a series of function values for each window.

- **`kelly_lateral(returns, n)`**: Computes the Kelly fraction for each column (ticker) in the returns DataFrame using `kelly_fraction`. If `n > 1`, applies a rolling mean over `n` periods to smooth the fractions; otherwise, returns the raw fractions. This provides lateral (cross-sectional) Kelly values across tickers.

In [ ]:

def kelly_fraction(returns):
        wins = returns[returns > 0]
        losses = returns[returns <= 0]
        
        W = len(wins) / len(returns) if len(returns) > 0 else 0
        avg_win = wins.mean() if len(wins) > 0 else 0
        avg_loss = abs(losses.mean()) if len(losses) > 0 else 0
        
        R = (avg_win / avg_loss) if avg_loss > 0 else 0
        kelly = max(W - (1 - W) / R,-1) if R != 0 else 0
        return kelly

def rolling_function(returns, function, f_duration):
    return returns.rolling(window=f_duration).apply(function, raw=False)

def kelly_lateral(returns, n):
        lateral_f = returns.apply(kelly_fraction, axis=1)
        if n > 1:
                kelly_lateral_df = lateral_f.rolling(n).mean()
        else:
                kelly_lateral_df = lateral_f
        return kelly_lateral_df

This code computes and visualizes Kelly fractions for position sizing using the Kelly Criterion.

- Calculates daily log returns from relative prices (`px_df_rel`).
- Initializes `kelly_df` with relative prices and signals.
- For each duration (33, 50, 100 days):
    - Applies rolling Kelly fraction to log returns.
    - Adds Kelly fractions to `kelly_df`.
    - Computes gross (sum of absolute Kelly values) and net (sum of Kelly values) exposures per duration.
- For each ticker, averages Kelly fractions across durations and plots against the signal.
- Computes average gross and net exposures across durations.
- Plots gross and net exposures over time.
- Prints `kelly_df` columns.

In [ ]:
daily_log_rets = np.log(px_df_rel / px_df_rel.shift(1))
kelly_df = pd.DataFrame(index=data.index)
kelly_df = pd.concat([px_df_rel.add_suffix('_rel'), score_rel.add_suffix('_signal')], axis=1)

f_duration_list = [33, 50, 100]
for f_duration in f_duration_list:
    print(f'value of 1 day for a duration of {f_duration} days: {round(1/f_duration, 2)} pct, ')
    kelly_log_df = rolling_function(daily_log_rets, kelly_fraction, f_duration)
    kelly_df = pd.concat([kelly_df, kelly_log_df.add_suffix(f'_f{f_duration}')], axis=1)
        
    kelly_gross = abs(kelly_log_df).sum(axis = 1)
    kelly_net = kelly_log_df.sum(axis = 1)    
    kelly_df[f'gross_{f_duration}'] = kelly_gross
    kelly_df[f'net_{f_duration}'] = kelly_net

for ticker in tickers_list:
    kelly_df[f'{ticker}_favg'] = kelly_df.filter(like=f'{ticker}_f').mean(axis=1)
    ax = kelly_df[[f'{ticker}_signal',f'{ticker}_favg']].dropna().plot(figsize=(15, 3), grid = True, secondary_y=[f'{ticker}_favg'], title=f'{ticker}: signal and Kelly average')
    ax.legend(loc='lower left')
    plt.show()
kelly_df['gross_favg'] =abs(kelly_df.filter(like=f'gross')).mean(axis=1)
kelly_df['net_favg'] = kelly_df.filter(like=f'net').mean(axis=1)

kelly_df.dropna().filter(like='gross').plot(figsize=(15, 3), grid=True, title = f'Gross Exposure - Kelly: {f_duration_list}')
kelly_df.dropna().filter(like='net').plot(figsize=(15, 3), grid=True, title = f'Net Exposure - Kelly: {f_duration_list}')

kelly_df.columns

The `simulate_kelly_position_sizing` function simulates a trading strategy using Kelly Criterion fractions for position sizing across multiple tickers. It iterates through each date in the dataset (starting from index 2), calculates target positions based on Kelly fractions (from `kelly_df`), executes trades with commissions and lot sizing, computes daily P&L, and tracks NAV, long market value, and short market value over time.

Key steps per date and ticker:
- Retrieve current and previous prices (`price_fx`, `price_fx_prev`).
- Set the signal as the rounded Kelly fraction for the ticker and duration (`pct`, e.g., 33, 50, 100, or 'avg').
- Compute the fraction for share calculation (based on `fraction_type`; here, it's 1, so `fraction = price_fx`).
- Calculate target market value as `nav * signal` (Kelly fraction as a direct percentage of NAV).
- Determine shares using `shares_target`, round to `lot_size`, and compute the difference from current position (`round_lots`).
- Calculate trade value and commission, update position and cash balance.
- Compute daily P&L (conditionally based on `look_ahead` flag) and accumulate total daily P&L.
- Track long/short exposures based on position sign.
- After processing all tickers, update NAV with total daily P&L and append to output lists.

Returns lists of NAV, long market values, and short market values for analysis. This assumes Kelly fractions are precomputed and stored in `kelly_df`, with no additional volatility adjustments in the fraction calculation.

In [ ]:
def simulate_kelly_position_sizing(start_K, lot_size, com_rate, pct, look_ahead,fraction_type, constant= None, ratio=None):
    # for pct in pct_list:
    nav = start_K
    cash_balance = start_K
    position_dict = {ticker: 0 for ticker in tickers_list}  # Tracks positions for each ticker
    pl_dict = {ticker: 0 for ticker in tickers_list}  # Tracks P&L for each ticker
    # daily_pl_sum = 0  # Tracks total daily P&L across all tickers
    nav_list = []  ;    long_mv_list = []   ;   short_mv_list = []   # Tracks NAV over time

    for t in data.index[2:]:  # Loop through each bar
        daily_pl_sum = 0 ; long_mv = 0 ; short_mv = 0  # Reset at every bar
        for ticker in tickers_list[:]:  # Loop through each ticker
            price_fx = data.at[t,f'{ticker}_fx']
            price_fx_prev = data[f'{ticker}_fx'].shift(1).at[t]
            # signal = data.at[t,f'{ticker}_signal']
            signal = kelly_df.at[t,f'{ticker}_f{pct}'].round(4)
            
            # Compute fraction based on type
            if fraction_type == 1:
                fraction = price_fx
            elif fraction_type == 2:
                fraction = price_fx * constant
            elif fraction_type == 3:
                fraction = price_fx *  data.at[t,f'{ticker}{ratio}']
            elif fraction_type == 4:
                fraction = data.at[t,f'{ticker}{ratio}']
            else:
                fraction = price_fx  # default to option 1

            # Calculate position size percentage,target market value and shares
            # target_mv = nav_signal(pct, nav,  signal)
            target_mv = np.multiply(nav, signal)
            shares = shares_target(target_mv, fraction)            
            
            # # Calculate daily P&L
            if not look_ahead:
                daily_pl = position_dict[ticker] * (price_fx - price_fx_prev)
                pl_dict[ticker] += daily_pl
                daily_pl_sum += daily_pl

                # Calculate theoretical lots and round lots:
            theoretical_lots = round_lot(shares, lot_size)
            round_lots = theoretical_lots - position_dict[ticker]

            # Calculate trade value and commission:
            trade_value = trading_value(round_lots, price_fx)
            commission = abs(trade_value) * com_rate

            # Update position and cash balance
            position_dict[ticker] += round_lots
            cash_balance -= (trade_value + commission)            
            
            # Calculate daily P&L
            if look_ahead:            
                daily_pl = position_dict[ticker] * (price_fx - price_fx_prev)
                pl_dict[ticker] += daily_pl
                daily_pl_sum += daily_pl
            
            # Calculate Long and Short exposure
            if position_dict[ticker] > 0:  # Long exposure
                long_mv += position_dict[ticker] * price_fx
            elif position_dict[ticker] < 0:  # Short exposure
                short_mv += position_dict[ticker] * price_fx

        # Update NAV
        nav += daily_pl_sum
        nav_list.append(nav)
        long_mv_list.append(long_mv)
        short_mv_list.append(short_mv)
    return nav_list, long_mv_list, short_mv_list

This code implements Kelly Criterion-based position sizing for a trading simulation across multiple tickers. It iterates over different Kelly fraction durations (from `f_duration_list`, e.g., 33, 50, 100 days) plus an average ('avg'), simulating portfolio performance for each. For each duration, it calls `simulate_kelly_position_sizing` to compute NAV, long/short market values, and exposures, then stores results in `nav_df`, calculates drawdowns, net exposure (using a potentially non-standard formula), and gross exposure. Finally, it plots NAV (with secondary y-axis for 'avg'), drawdowns, gross exposure, and net exposure for comparison across durations. The plots use `annual_target_list` in titles, though it's unrelated to Kelly here.

In [ ]:
pct_list = f_duration_list +['avg']

algo = 'kelly'
nav_cols = [f'nav_fix_pct{pct}' for pct in pct_list]
gross_exposure_cols = [f'gross_{algo}_{pct}' for pct in pct_list]
net_exposure_cols = [f'net_{algo}_{pct}' for pct in pct_list]

for pct in pct_list:
    nav_list, long_mv_list, short_mv_list = simulate_kelly_position_sizing(start_K, lot_size, 
                                                                           com_rate, pct, look_ahead,fraction_type =1, constant= None, ratio=None)

    nav_df[f'nav_{algo}_{pct}'] = nav_list
    nav_df[f'longMV_{algo}_{pct}'] = long_mv_list
    nav_df[f'shortMV_{algo}_{pct}'] = short_mv_list

    nav_df[f'drawdown_{algo}_{pct}'] = calculate_drawdown(nav_df[f'nav_{algo}_{pct}'])
    nav_df[f'net_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] + nav_df[f'shortMV_{algo}_{pct}']) / (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) 
    nav_df[f'gross_{algo}_{pct}'] = (nav_df[f'longMV_{algo}_{pct}'] - nav_df[f'shortMV_{algo}_{pct}']) / nav_df[f'nav_{algo}_{pct}']

nav_df.dropna().filter(like=f'nav_{algo}').plot(figsize=(15, 5), 
                            grid=True, secondary_y = [f'nav_{algo}_{pct}'], title=f'NAV - {algo}: {annual_target_list}')
nav_df.dropna().filter(like=f'drawdown_{algo}').plot(figsize=(15,3), grid=True, title = f'Drawdowns - {algo}: {annual_target_list}')
nav_df.dropna().filter(like=f'gross_{algo}').plot(figsize=(15, 3), grid=True, title = f'Gross Exposure - {algo}: {annual_target_list}')
nav_df.dropna().filter(like=f'net_{algo}').plot(figsize=(15, 3), grid=True, title = f'Net Exposure - {algo}: {annual_target_list}')

In [ ]:
nav_df = remove_duplicates(nav_df, keeping='last')

This code generates comparative plots for six position sizing algorithms ('fix_pct', 'fix_risk', 'raw_vol', 'std', 'atr', 'kelly') across NAV, drawdowns, and gross exposures. It loops over four parameter levels (i=0 to 3), selecting the i-th column for each algorithm's metric from `nav_df`, dropping NaN values, and plotting with a 15x5 figure size, grid, and titles. For NAV plots, the last series (Kelly) uses a secondary y-axis; secondary y-axis is commented out for drawdowns and gross exposures. This visualizes performance trade-offs across algorithms at each level.

In [ ]:
algo_list =['fix_pct', 'fix_risk', 'raw_vol', 'std', 'atr', 'kelly']

for i in range (4):
    plot_nav_cols = [nav_df.filter(like=f'nav_{algo}').columns[i] for algo in algo_list]
    nav_df[plot_nav_cols].dropna().plot(figsize=(15, 5), secondary_y=plot_nav_cols[-1:], 
                                        grid=True, title=f'NAV Comparison {i+1}: {algo_list}')
    plot_dd_cols = [nav_df.filter(like=f'drawdown_{algo}').columns[i] for algo in algo_list]
    nav_df[plot_dd_cols].dropna().plot(figsize=(15, 5), #secondary_y=plot_dd_cols[-1:], 
                                       grid=True, title=f'Drawdown Comparison {i+1}: {algo_list}')
    plot_gross_cols = [nav_df.filter(like=f'gross_{algo}').columns[i] for algo in algo_list]
    nav_df[plot_gross_cols].dropna().plot(figsize=(15, 5),# secondary_y=plot_gross_cols[-1:], 
                                          grid=True, title=f'Gross Comparison {i+1}: {algo_list}')